# Case Study: BioPharma

### Importing Required Libraries

In [1]:
import pandas as pd  
import pulp
import numpy as np

### Reading and Entering the Input Data

#### Table 6-18: Sales (Demand)

In [2]:
data_demand = pd.read_excel('RSM_2405-SCM_2024-Biopharma Model.xlsx', skiprows = 5, nrows=7,  usecols= 'A:C')
data_demand.rename(columns={"Unnamed: 0": "Market"}, inplace=True)
display(data_demand)

,Market,Highcal,Relax
0,Latin America,7,7
1,Europe,15,12
2,Asia w/o Japan,5,3
3,Japan,7,8
4,Mexico,3,3
5,U.S.,18,17


In [3]:
np.random.normal(7, 2, 1)

array([6.69590379])

In [4]:
# Create list of years in the future to account for
years = [i for i in range(2006, 2014)]
years

[2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013]

In [5]:
# Create a dictionary to store the demand for each nextyear
demands = {}
for i in range(len(years)):
    d = data_demand.copy()
    # Replace values of Asia w/o Japan with forcasted decrease
    d.iloc[2, 1:] = d.iloc[2, 1:]*pow(0.98,7-i)
    demands[years[i]] = d

demands[2013] = data_demand

In [6]:
demands[2013]

,Market,Highcal,Relax
0,Latin America,7,7
1,Europe,15,12
2,Asia w/o Japan,5,3
3,Japan,7,8
4,Mexico,3,3
5,U.S.,18,17


In [7]:
demands[2012].iloc[:4, 1:] = demands[2012].iloc[:4, 1:]*0

In [8]:
demands[2012]

,Market,Highcal,Relax
0,Latin America,0.0,0.0
1,Europe,0.0,0.0
2,Asia w/o Japan,0.0,0.0
3,Japan,0.0,0.0
4,Mexico,3.0,3.0
5,U.S.,18.0,17.0


#### Table 6-18: Capacity

In [9]:
data_capacity = pd.read_excel('RSM_2405-SCM_2024-Biopharma Model.xlsx', skiprows = 4, nrows=7,  usecols= 'D:E')
data_capacity = data_capacity.iloc[1:]
display(data_capacity)

,Plant,Capacity
1,Brazil,18.0
2,Germany,45.0
3,India,18.0
4,Japan,10.0
5,Mexico,30.0
6,U.S.,22.0


#### Table 6-19: Fixed and Variable Procution Cost in 2013 (US $)

In [10]:
data_fixed_variable_cost = pd.read_excel('RSM_2405-SCM_2024-Biopharma Model.xlsx', skiprows = 22, nrows=7,  usecols= 'A:H')
display(data_fixed_variable_cost)

,Plant,Plant Fixed Cost (Million $),Highcal Fixed Cost (Million $),Relax Fixed Cost (Million $),Highcal Raw Material Cost ($/kg),Highcal Production Cost ($/kg),Relax Raw Material Cost ($/kg),Relax Production Cost ($/kg)
0,Brazil,20,5,5,3.6,5.1,4.6,6.6
1,Germany,45,13,13,3.9,6.0,5.0,7.0
2,India,14,3,3,3.6,4.5,4.5,6.0
3,Japan,13,4,4,3.9,6.0,5.1,7.0
4,Mexico,30,6,6,3.6,5.0,4.6,6.5
5,U.S.,23,5,5,3.6,5.0,4.5,6.5


#### Table 6-20: Transporation Costs from Plants to Markets (US $/kg)

In [11]:
data_transportation_cost = pd.read_excel('RSM_2405-SCM_2024-Biopharma Model.xlsx', skiprows = 83, nrows=7,  usecols= 'A:G')
display(data_transportation_cost)

,From / To,Latin America,Europe,Asia w/o Japan,Japan,Mexico,U.S.
0,Brazil,0.20,0.45,0.50,0.50,0.40,0.45
1,Germany,0.45,0.20,0.35,0.40,0.30,0.30
2,India,0.50,0.35,0.20,0.30,0.50,0.45
3,Japan,0.50,0.40,0.30,0.10,0.45,0.45
4,Mexico,0.40,0.30,0.50,0.45,0.20,0.25
5,U.S.,0.45,0.30,0.45,0.45,0.25,0.20


#### Table 6-21 History of Exhange Rates

In [12]:
data = {'Currency': ['Brazilian Real', 'Euro', 'Indian Rupee', 'Japanese Yen', 'Mexican Peso', 'U.S. Dollar'],
       '2013': [2.15, 0.75, 58.44, 97.58, 12.75, 1],
       '2012': [1.95, 0.78, 53.46, 79.79, 13.15, 1.00],
       '2011': [1.67, 0.72, 46.85, 79.70, 12.42, 1.00],
       '2010':[1.75, 0.75, 45.72, 87.78, 12.63, 1.00],
       '2009':[1.99, 0.72, 48.42, 93.58, 13.48, 1.00],
       '2008':[1.83, 0.68, 43.62, 103.42, 11.13, 1.00],
       '2007':[1.94, 0.73, 41.34, 117.77, 10.92, 1.00],
       '2006':[2.17, 0.80, 45.18, 116.29, 10.89, 1.00]}
 
data_exchange_rate = pd.DataFrame(data)
display(data_exchange_rate) # Table 6-21

,Currency,2013,2012,2011,2010,2009,2008,2007,2006
0,Brazilian Real,2.15,1.95,1.67,1.75,1.99,1.83,1.94,2.17
1,Euro,0.75,0.78,0.72,0.75,0.72,0.68,0.73,0.80
2,Indian Rupee,58.44,53.46,46.85,45.72,48.42,43.62,41.34,45.18
3,Japanese Yen,97.58,79.79,79.70,87.78,93.58,103.42,117.77,116.29
4,Mexican Peso,12.75,13.15,12.42,12.63,13.48,11.13,10.92,10.89
5,U.S. Dollar,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00


#### Table 6-22 Import Tariffs

In [13]:
data_duties = pd.read_excel('RSM_2405-SCM_2024-Biopharma Model.xlsx', skiprows = 15, nrows=2,  usecols= 'A:F')
display(data_duties)

,Latin America,Europe,Asia w/o Japan,Japan,Mexico,U.S.
0,0.3,0.03,0.27,0.06,0.35,0.04


In [14]:
n = 6 # number of plants
m = 6 # number of markets
S = 8 # number of scenarios (years from 2006-2013)

### Calculation of Fixed and Variable Costs for a Specific Year

In [15]:
# Create a function to get the dataframe with fixed costs given the exchange rates for the year
def get_costs(cost_df, Year):
    """
    
        Parameters:
            - cost_df (DataFrame): Dataframe containing cost data
            - Year (int): year of scenario exchange rates
            - fixed (boolean): 1 to return fixed costs, 0 to return variable costs
    """
    mod_costs = cost_df.copy()
    cols = len(mod_costs.columns) - 1
      
    for i in range(0,n):
        for q in range(0, cols):
            mod_costs.iloc[i,1+q] = (mod_costs.iloc[i][1+q]*data_exchange_rate.iloc[i,1])/data_exchange_rate.iloc[i,1 + 2013 -Year]

    return mod_costs.iloc[:, :4]

In [16]:
# Create a dictionary to store the modified_fixed_variable cost for each scenario (year)
years = [i for i in range(2006, 2014)]
mod_fvc = { year: get_costs(data_fixed_variable_cost, year) for year in years }

In [17]:
# Show example scenario costs
mod_fvc[2012]

,Plant,Plant Fixed Cost (Million $),Highcal Fixed Cost (Million $),Relax Fixed Cost (Million $)
0,Brazil,22.051282,5.512821,5.512821
1,Germany,43.269231,12.500000,12.500000
2,India,15.304153,3.279461,3.279461
3,Japan,15.898484,4.891841,4.891841
4,Mexico,29.087452,5.817490,5.817490
5,U.S.,23.000000,5.000000,5.000000


### Highcal Variable Costs for a Specific Year

In [18]:
variable_cost_highcal = data_transportation_cost.copy()

for i in range(0,n):
    for j in range(0,m):
        variable_cost_highcal.iloc[i,1+j] = data_transportation_cost.iloc[i,1+j] + data_fixed_variable_cost.iloc[i,4] + data_fixed_variable_cost.iloc[i,5]

# display(np.round(variable_cost_highcal,2))

### Relax Variable Costs for a Specific Year

In [19]:
variable_cost_relax = data_transportation_cost.copy()

for i in range(0,n):
    for j in range(0,m):
        variable_cost_relax.iloc[i,1+j] = data_transportation_cost.iloc[i,1+j] + data_fixed_variable_cost.iloc[i,6] + data_fixed_variable_cost.iloc[i,7]

# display(np.round(variable_cost_relax,2))

### Highcal Total Variable Costs Including Duties for a Specific Year

In [20]:
total_variable_cost_highcal = variable_cost_highcal.copy()

for i in range(0,n):
    for j in range(0,m):
        if i!=j: # No need to pay for import duty when i=j (e.g., Brazil to Latin America)
            total_variable_cost_highcal.iloc[i,1+j] = total_variable_cost_highcal.iloc[i,1+j]*(1+data_duties.iloc[0,j])
            
display(np.round(total_variable_cost_highcal,2))

,From / To,Latin America,Europe,Asia w/o Japan,Japan,Mexico,U.S.
0,Brazil,8.90,9.42,11.68,9.75,12.28,9.52
1,Germany,13.46,10.10,13.02,10.92,13.77,10.61
2,India,11.18,8.70,8.30,8.90,11.61,8.89
3,Japan,13.52,10.61,12.95,10.00,13.97,10.76
4,Mexico,11.70,9.17,11.56,9.59,8.80,9.20
5,U.S.,11.76,9.17,11.49,9.59,11.95,8.80


### Relax Total Variable Costs Including Duties for a Specific Year

In [21]:
total_variable_cost_relax = variable_cost_relax.copy()

for i in range(0,n):
    for j in range(0,m):
        if i!=j:
            total_variable_cost_relax.iloc[i,1+j] = total_variable_cost_relax.iloc[i,1+j]*(1+data_duties.iloc[0,j])
            
display(np.round(total_variable_cost_relax,2))

,From / To,Latin America,Europe,Asia w/o Japan,Japan,Mexico,U.S.
0,Brazil,11.40,12.00,14.86,12.40,15.66,12.12
1,Germany,16.18,12.20,15.68,13.14,16.60,12.79
2,India,14.30,11.18,10.70,11.45,14.85,11.39
3,Japan,16.38,12.88,15.75,12.20,16.94,13.05
4,Mexico,14.95,11.74,14.73,12.24,11.30,11.80
5,U.S.,14.88,11.64,14.54,12.14,15.19,11.20


### New Network Design Model

##### Parameters

- $n$ = number of potential plant locations/capacity
- $m$ = number of markets or demand points
- $k=1$ refers to Highcal and $k=2$ refers to Relax 
- $D_{jks}$ = annual demand from market $j$ for product $k$ 
- $L_{i}$ = potential capacity of plant $i$
- $g_{is}$ = fixed cost of keeping plant $i$ open in scenario $s$
- $f_{iks}$ = fixed cost of keeping plant $i$ open and producing product $k$ in scenario $s$
- $c_{ijks}$ = cost of producing and shipping one unit of product $k$ from plant $i$ to market $j$ in scenario $s$
- $s$ = scenario number given a set of exchange rates for a year
- $p_{s}$ = 0.125 = probability of scenario $s$ given equal weighting

##### Decision Variables
- $y_{is}$ = 1 to keep plant i, 0 to shut down plant $i$ in scenario $s$
- $h_{is}$ = 1 to produce Highcal at plant i, 0 to not produce Highcal at plant $i$ in scenario $s$
- $r_{is}$ = 1 to produce Relax at plant i, 0 to not produce Relax at plant $i$ in scenario $s$
- $x_{ijks}=$ quantity of product $k$ shipped from plant $i$ to market $j$ in scenario $s$

\begin{equation}
\begin{aligned}
\min_{z,y,x} \quad & \sum_{s=1}^{S}p_{s} \biggl(0.2\sum_{i=1}^{n}(g_{is} +\sum_{k=1}^{2} f_{iks}) + 0.8\sum_{i=1}^{n}(g_{is}y_{is} +f_{i1s}h_{is} + f_{i2s}r_{is}) + \sum_{i=1}^{n}\sum_{j=1}^{m}\sum_{k=1}^{2} c_{ijks}x_{ijks}\biggr) \\
\textrm{s.t.} \quad & \sum_{i=1}^{n} x_{ijks}=D_{jks} && \forall  j=1,\dots,m; k=1,2; s=1,\dots,S\\
  &\sum_{j=1}^{m}\sum_{k=1}^2 x_{ijks} \leq L_{i}y_{is} && \forall i=1,\dots,n;s=1,\dots,S \\
  & \sum_{j=1}^{m}x_{ij1s} \leq L_{i}h_{is} && \forall i=1,\dots,n;s=1,\dots,S \\
  & \sum_{j=1}^{m}x_{ij2s} \leq L_{i}r_{is} && \forall i=1,\dots,n;s=1,\dots,S \\
  & x_{ijks} \geq 0 && \forall i=1,\dots,n; j=1,\dots,m;k=1,2;s=1,\dots,S \\
  & y_{is}, h_{is}, r_{is} \in \{0, 1\} && \forall i=1,\dots,n;s=1,\dots,S \\
  & y_{is} \geq y_{i(s-1)} && \forall i=1,\dots,n;s=1,\dots,S \\
  & h_{is}, r_{is} \leq y_{is} && \forall i=1,\dots,n;s=1,\dots,S \\
\end{aligned}
\end{equation}

### Decision variables

In [22]:
# Create x_ijk decision variables
x = pulp.LpVariable.dicts("quantity_shipped", ((i,j,k,s) for i in range(0,n) for j in range(0,m) for k in range(1,3) for s in range(0,S)), lowBound=0.0, cat='continuous')

In [23]:
# Create y_i decision variables
y = pulp.LpVariable.dicts("open_or_close", ((i, s) for i in range(0,n) for s in range(0,S)), cat='Binary')

In [24]:
# Create h_i decision variables
h = pulp.LpVariable.dicts("prod_H", ((i, s) for i in range(0,n) for s in range(0,S)), cat='Binary')

In [25]:
# Create r_i decision variables
r = pulp.LpVariable.dicts("prod_R", ((i, s) for i in range(0,n) for s in range(0,S)), cat='Binary')

### Model Initialization

In [26]:
model_biopharma = pulp.LpProblem(name="Model_Biopharma", sense=pulp.LpMinimize)

### Constraints

In [27]:
# Set demand fullfillment constraints
for s in range(0,S):
    for j in range(0,m):
        model_biopharma += (pulp.lpSum(x[i, j, 1, s] for i in range(0,n)) == demands[2006+s].iloc[j][1])
        model_biopharma += (pulp.lpSum(x[i, j, 2, s] for i in range(0,n)) == demands[2006+s].iloc[j][2])

In [28]:
# Set total capacity constraints
for s in range(0,S):
    for i in range(0,n):
        model_biopharma += (pulp.lpSum(x[i, j, k, s] for j in range(0,m) for k in range(1,3)) <= data_capacity.iloc[i][1]*y[i,s])
        model_biopharma += (pulp.lpSum(x[i, j, 1, s] for j in range(0,m)) <= data_capacity.iloc[i][1]*h[i,s])
        model_biopharma += (pulp.lpSum(x[i, j, 2, s] for j in range(0,m)) <= data_capacity.iloc[i][1]*r[i,s])

In [29]:
# Set constraints for h_i and r_i variables
for s in range(0, S):
    for i in range(0,n):
        if (s == 0):
            model_biopharma += h[i,s] <= y[i,s]
            model_biopharma += r[i,s] <= y[i,s]
        else:
            model_biopharma += y[i,s] >= y[i,s-1]
            model_biopharma += h[i,s] <= y[i,s]
            model_biopharma += r[i,s] <= y[i,s]

### Objective

In [30]:
# Create a dictionary to store the total fixed cost, highcal var cost, relax var cost and total var cost
totals = {'fc': 0, 'hvc': 0, 'rvc': 0, 'tvc': 0}

In [31]:
# Create a function to get the cost for a given scenario
def get_Total_Cost(s, solution, totals=totals): 
    """
        Parameters:
            - Year:
            - solution (boolean): whether or not to use function to get solution costs
            - totals (dictionary{str:int}): dictionary to store cost totals
    """
    
    if solution:
        
        # Calculate the costs
        Fixed_Cost_1 = 0.2*pulp.lpSum(mod_fvc[2006+s].iloc[i, 1] + mod_fvc[2006+s].iloc[i, 2] + mod_fvc[2006+s].iloc[i, 3] for i in range(0,n)) 
        Fixed_Cost_2 = 0.8*pulp.lpSum(mod_fvc[2006+s].iloc[i, 1]*y[i,s].varValue + (mod_fvc[2006+s].iloc[i, 2]*h[i,s].varValue) + (mod_fvc[2006+s].iloc[i, 3]*r[i,s].varValue) for i in range(0,n)) 
        Variable_Cost_Highcal = pulp.lpSum(total_variable_cost_highcal.iloc[i][j+1]*x[i, j, 1, s].varValue for i in range(0,n) for j in range(0,m))
        Variable_Cost_Relax = pulp.lpSum(total_variable_cost_relax.iloc[i][j+1]*x[i, j, 2, s].varValue for i in range(0,n) for j in range(0,m))
    
        # Add the costs to the total cost variables
        totals['fc'] += Fixed_Cost_1 + Fixed_Cost_2
        totals['hvc'] += Variable_Cost_Highcal
        totals['rvc'] += Variable_Cost_Relax
        totals['tvc'] += Variable_Cost_Highcal + Variable_Cost_Relax
    
    else:
        Fixed_Cost_1 = 0.2*pulp.lpSum(mod_fvc[2006+s].iloc[i, 1] + mod_fvc[2006+s].iloc[i, 2] + mod_fvc[2006+s].iloc[i, 3] for i in range(0,n)) 
        Fixed_Cost_2 = 0.8*pulp.lpSum(mod_fvc[2006+s].iloc[i, 1]*y[i,s] + (mod_fvc[2006+s].iloc[i, 2]*h[i,s]) + (mod_fvc[2006+s].iloc[i, 3]*r[i,s]) for i in range(0,n)) 
        Variable_Cost_Highcal = pulp.lpSum(total_variable_cost_highcal.iloc[i][j+1]*x[i, j, 1, s] for i in range(0,n) for j in range(0,m))
        Variable_Cost_Relax = pulp.lpSum(total_variable_cost_relax.iloc[i][j+1]*x[i, j, 2, s] for i in range(0,n) for j in range(0,m))

    Total_Cost = Fixed_Cost_1 + Fixed_Cost_2 + Variable_Cost_Highcal + Variable_Cost_Relax
    
    return Total_Cost

In [32]:
# Get the total cost summed over all scenarios and add to model
p = 1/8
all_costs = pulp.lpSum(p*get_Total_Cost(s, False) for s in range(0,S))
model_biopharma += all_costs
model_biopharma.writeLP("model_biopharma.lp")

[open_or_close_(0,_0),
 open_or_close_(0,_1),
 open_or_close_(0,_2),
 open_or_close_(0,_3),
 open_or_close_(0,_4),
 open_or_close_(0,_5),
 open_or_close_(0,_6),
 open_or_close_(0,_7),
 open_or_close_(1,_0),
 open_or_close_(1,_1),
 open_or_close_(1,_2),
 open_or_close_(1,_3),
 open_or_close_(1,_4),
 open_or_close_(1,_5),
 open_or_close_(1,_6),
 open_or_close_(1,_7),
 open_or_close_(2,_0),
 open_or_close_(2,_1),
 open_or_close_(2,_2),
 open_or_close_(2,_3),
 open_or_close_(2,_4),
 open_or_close_(2,_5),
 open_or_close_(2,_6),
 open_or_close_(2,_7),
 open_or_close_(3,_0),
 open_or_close_(3,_1),
 open_or_close_(3,_2),
 open_or_close_(3,_3),
 open_or_close_(3,_4),
 open_or_close_(3,_5),
 open_or_close_(3,_6),
 open_or_close_(3,_7),
 open_or_close_(4,_0),
 open_or_close_(4,_1),
 open_or_close_(4,_2),
 open_or_close_(4,_3),
 open_or_close_(4,_4),
 open_or_close_(4,_5),
 open_or_close_(4,_6),
 open_or_close_(4,_7),
 open_or_close_(5,_0),
 open_or_close_(5,_1),
 open_or_close_(5,_2),
 open_or_cl

In [33]:
# solver = pulp.PULP_CBC_CMD(msg=0) # msg=0 to disable the log of calculation

## Gurobi
# Gurobi academic license available at https://www.gurobi.com/academia/academic-program-and-licenses/

# Use the following command to install Gurobi
# conda install -c gurobi gurobi=10.0.1

solver = pulp.GUROBI_CMD()

## Cplex
# solver = pulp.CPLEX_CMD()

In [34]:
model_biopharma.solve(solver)

1

In [35]:
# Show the total costs
all_costs = pulp.lpSum(p*get_Total_Cost(s, True) for s in range(0,S))
print("Fixed Cost =  " + str(p*totals['fc']))
print("Highcal Variable Cost: " + str(p*totals['hvc']))
print("Relax Variable Cost: " + str(p*totals['rvc']))
print("Total Variable Cost: " + str(p*totals['tvc']))
print("Total Cost: " + str(all_costs))

Fixed Cost =  198.20072693849608
Highcal Variable Cost: 453.4947160178457
Relax Variable Cost: 534.9106342795898
Total Variable Cost: 988.4053502974353
Total Cost: 1186.6060772359313


In [36]:
# Get the solutions for y_is, h_is, and r_is
sol_ys, sol_hs, sol_rs = {}, {}, {}

for s in range(S):
    sol_ys[s] = [y[i,s].varValue for i in range(n)]
    sol_hs[s] = [h[i,s].varValue for i in range(n)]
    sol_rs[s] = [r[i,s].varValue for i in range(n)]

In [37]:
# Show the solution for each scenario
plants = ['Brazil', 'Germany', 'India','Japan', 'Mexico', 'US']
for s in range(S):
    print("Scenario {}, year: {}".format(s, 2006+s))
    solution = pd.DataFrame({'Plant':plants, 'Open/Close': sol_ys[s], 
                             'Produce HighCal': sol_hs[s], 'Produce Relax': sol_rs[s]})
    display(solution)

Scenario 0, year: 2006


,Plant,Open/Close,Produce HighCal,Produce Relax
0,Brazil,1.0,1.0,1.0
1,Germany,1.0,0.0,1.0
2,India,1.0,1.0,1.0
3,Japan,0.0,0.0,0.0
4,Mexico,1.0,1.0,1.0
5,US,1.0,1.0,1.0


Scenario 1, year: 2007


,Plant,Open/Close,Produce HighCal,Produce Relax
0,Brazil,1.0,1.0,1.0
1,Germany,1.0,0.0,1.0
2,India,1.0,1.0,1.0
3,Japan,0.0,0.0,0.0
4,Mexico,1.0,1.0,1.0
5,US,1.0,1.0,1.0


Scenario 2, year: 2008


...

,Plant,Open/Close,Produce HighCal,Produce Relax
0,Brazil,1.0,1.0,1.0
1,Germany,1.0,0.0,1.0
2,India,1.0,1.0,1.0
3,Japan,0.0,0.0,0.0
4,Mexico,1.0,1.0,1.0
5,US,1.0,1.0,1.0


Scenario 6, year: 2012


,Plant,Open/Close,Produce HighCal,Produce Relax
0,Brazil,1.0,0.0,0.0
1,Germany,1.0,0.0,0.0
2,India,1.0,1.0,0.0
3,Japan,0.0,0.0,0.0
4,Mexico,1.0,1.0,1.0
5,US,1.0,0.0,1.0


Scenario 7, year: 2013


,Plant,Open/Close,Produce HighCal,Produce Relax
0,Brazil,1.0,1.0,1.0
1,Germany,1.0,0.0,1.0
2,India,1.0,1.0,1.0
3,Japan,0.0,0.0,0.0
4,Mexico,1.0,1.0,1.0
5,US,1.0,1.0,1.0


In [38]:
# Create a function to output the solution x_ijkt for a given t
def show_solution(t):
    Solution_highcal = []
    for i in range(0,n):
            Solution_highcal.append({'Latin America': x[(i,0,1,t)].varValue,
                             'Europe': x[(i,1,1,t)].varValue,
                             'Asia w/o Japan': x[(i,2,1,t)].varValue,
                             'Japan': x[(i,3,1,t)].varValue,
                             'Mexico': x[(i,4,1,t)].varValue,
                             'U.S.': x[(i,5,1,t)].varValue})

    Solution_relax = []
    for i in range(0,n):
            Solution_relax.append({'Latin America': x[(i,0,2,t)].varValue,
                             'Europe': x[(i,1,2,t)].varValue,
                             'Asia w/o Japan': x[(i,2,2,t)].varValue,
                             'Japan': x[(i,3,2,t)].varValue,
                             'Mexico': x[(i,4,2,t)].varValue,
                             'U.S.': x[(i,5,2,t)].varValue})


    quantity_highcal = pd.DataFrame(Solution_highcal)
    quantity_highcal.insert(0, "From\To", ['Brazil', 'Germany', 'India','Japan', 'Mexico', 'US'])

    quantity_relax = pd.DataFrame(Solution_relax)
    quantity_relax.insert(0, "From\To", ['Brazil', 'Germany', 'India','Japan', 'Mexico', 'US'])


    print("Optimal Highcal Quantity (Million Kilograms):")
    display(quantity_highcal)

    print("Optimal Relax Quantity (Million Kilograms):")
    display(quantity_relax)

In [39]:
# Use for loop to show solution for each year
for s in range(0,S):
    print("x_ijks for t = {}: Year {}".format(s, 2006+s))
    show_solution(s)
    print("\n")

x_ijks for t = 0: Year 2006
Optimal Highcal Quantity (Million Kilograms):


,From\To,Latin America,Europe,Asia w/o Japan,Japan,Mexico,U.S.
0,Brazil,7.0,4.0,0.000000,0.0,0.0,0.0
1,Germany,0.0,0.0,0.000000,0.0,0.0,0.0
2,India,0.0,0.0,4.340628,7.0,0.0,0.0
3,Japan,0.0,0.0,0.000000,0.0,0.0,0.0
4,Mexico,0.0,11.0,0.000000,0.0,3.0,13.0
5,US,0.0,0.0,0.000000,0.0,0.0,5.0


Optimal Relax Quantity (Million Kilograms):


,From\To,Latin America,Europe,Asia w/o Japan,Japan,Mexico,U.S.
0,Brazil,7.0,0.0,0.000000,0.000000,0.0,0.0
1,Germany,0.0,12.0,0.000000,3.945004,0.0,0.0
2,India,0.0,0.0,2.604377,4.054996,0.0,0.0
3,Japan,0.0,0.0,0.000000,0.000000,0.0,0.0
4,Mexico,0.0,0.0,0.000000,0.000000,3.0,0.0
5,US,0.0,0.0,0.000000,0.000000,0.0,17.0




x_ijks for t = 1: Year 2007
Optimal Highcal Quantity (Million Kilograms):


...

,From\To,Latin America,Europe,Asia w/o Japan,Japan,Mexico,U.S.
0,Brazil,0.0,0.0,0.0,0.0,0.0,0.0
1,Germany,0.0,0.0,0.0,0.0,0.0,0.0
2,India,0.0,0.0,0.0,0.0,0.0,0.0
3,Japan,0.0,0.0,0.0,0.0,0.0,0.0
4,Mexico,0.0,0.0,0.0,0.0,3.0,0.0
5,US,0.0,0.0,0.0,0.0,0.0,17.0




x_ijks for t = 7: Year 2013
Optimal Highcal Quantity (Million Kilograms):


,From\To,Latin America,Europe,Asia w/o Japan,Japan,Mexico,U.S.
0,Brazil,7.0,4.0,0.0,0.0,0.0,0.0
1,Germany,0.0,0.0,0.0,0.0,0.0,0.0
2,India,0.0,0.0,5.0,7.0,0.0,0.0
3,Japan,0.0,0.0,0.0,0.0,0.0,0.0
4,Mexico,0.0,11.0,0.0,0.0,3.0,13.0
5,US,0.0,0.0,0.0,0.0,0.0,5.0


Optimal Relax Quantity (Million Kilograms):


,From\To,Latin America,Europe,Asia w/o Japan,Japan,Mexico,U.S.
0,Brazil,7.0,0.0,0.0,0.0,0.0,0.0
1,Germany,0.0,12.0,0.0,5.0,0.0,0.0
2,India,0.0,0.0,3.0,3.0,0.0,0.0
3,Japan,0.0,0.0,0.0,0.0,0.0,0.0
4,Mexico,0.0,0.0,0.0,0.0,3.0,0.0
5,US,0.0,0.0,0.0,0.0,0.0,17.0
